In [1]:
import os
os.chdir("../")



In [2]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

from src.params.api_params import *
from src.data_gathering import APIRequest

In [3]:
api_request = APIRequest()
data = api_request.read_data()
data.head()

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,7d_sell_sold,1d_buy_sold,1d_sell_sold,lastUpdate
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-24T08:00:01.483Z
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
3,24,Sealed Package of Snowballs,Consumable,Basic,82968.0,88.0,153151.0,127.0,1025.0,2497.0,97.0,542.0,2024-08-09T11:48:10.986Z
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-11T18:00:01.806Z


In [4]:
# transformaciones

# daily: es el 0.05 de 1d_sell_sold

# en este caso te arriesgas cuando hay un pico en un dia de venta. lo que se puede hacer es comparar este valor con el 7d_sell_sold a ver como de proporcional es

# una idea es ver el 7d_sell_sold / 7 para ver el promedio por día de esa semana. Y por ejemplo escoger el valor mas pequeño entre este y el daily de vortus

# dice de buscar un item que el 1d_sell_sold sea 7 veces menor que el 7d_sell_sold

# profit: sell_price * 0.085 - buy_price

# profit_total = profit * daily

# ROI: (sell_price * 0.85 - buy_price)/buy_price * 100

In [5]:
data_transformed = api_request.transform_data(data)
data_transformed

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,7d_sell_sold,...,1d_sell_sold,lastUpdate,daily,1d_sell_sold_avg,1d_lower_than_7d_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,ROI
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-09-24T08:00:01.483Z,0,0,False,0.0,0,0.0,0.0,0.0000
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
3,24,Sealed Package of Snowballs,Consumable,Basic,82968.0,88.0,153151.0,127.0,1025.0,2497.0,...,542.0,2024-08-09T11:48:10.986Z,27,356,False,356.0,17,20.0,340.0,0.2267
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2023-05-11T18:00:01.806Z,0,0,False,0.0,0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67141,102182,Elonian Bazaar Shade,Consumable,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2024-07-16T19:00:05.426Z,0,0,False,0.0,0,0.0,0.0,0.0000
67142,102183,Sandswept Pistol,Weapon,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2024-07-30T19:00:04.532Z,0,0,False,0.0,0,0.0,0.0,0.0000
67143,102184,Corrupted Hero Pistol,Weapon,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2024-07-18T01:00:04.294Z,0,0,False,0.0,0,0.0,0.0,0.0000
67144,102185,Corrupted Hero Weapon Locker,Container,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2024-07-16T17:00:17.343Z,0,0,False,0.0,0,0.0,0.0,0.0000


In [7]:
# filtros
# el 1day sold menor que una septima parte de 7d sold
data_filtered = data_transformed[data_transformed['1d_lower_than_7d_avg'] == True]

# daily_mod > 0
data_filtered = data_filtered[data_filtered['daily_mod'] > DAILY_MIN]
# ROI > 10
data_filtered = data_filtered[data_filtered['ROI'] > ROI_MIN]
data_filtered = data_filtered[data_filtered['ROI'] < ROI_MAX]

# profit > 10 platas (1000)
PROFIT_MIN
data_filtered = data_filtered[data_filtered['profit'] >= PROFIT_MIN]

# ordenacion

data_filtered = data_filtered.sort_values('ROI', ascending=False)

data_filtered

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,7d_sell_sold,...,1d_sell_sold,lastUpdate,daily,1d_sell_sold_avg,1d_lower_than_7d_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,ROI
20752,28583,Dragonfury,Weapon,Exotic,935.0,4819.0,311.0,10697.0,356.0,352.0,...,42.0,2024-08-09T11:48:20.575Z,2,50,True,42.0,2,4273.0,8546.0,0.8868
20743,28574,Berserker's Iron Spear of Agony,Weapon,Exotic,1070.0,4495.0,321.0,9899.0,644.0,473.0,...,66.0,2024-08-09T11:48:20.568Z,3,67,True,66.0,3,3919.0,11757.0,0.8719
31589,45027,Twisted Watchwork Portal Device,Consumable,Exotic,2616.0,1233.0,4502.0,2650.0,208.0,942.0,...,60.0,2024-08-09T11:48:22.823Z,3,134,True,60.0,3,1020.0,3060.0,0.8268
53371,83502,Superior Rune of the Renegade,UpgradeComponent,Exotic,2057.0,2193.0,3115.0,4498.0,569.0,755.0,...,62.0,2024-08-09T11:48:25.770Z,3,107,True,62.0,3,1630.0,4890.0,0.7434
14674,19880,Carrion Intricate Silk Insignia,CraftingMaterial,Rare,2216.0,2489.0,1308.0,5096.0,3040.0,859.0,...,105.0,2024-08-09T11:48:16.262Z,5,122,True,105.0,5,1843.0,9215.0,0.7403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34187,47873,Soldier's Benthic Hydrobreather of the Afflicted,Armor,Rare,1410.0,22159.0,828.0,28889.0,145.0,199.0,...,23.0,2024-08-09T11:48:23.667Z,1,28,True,23.0,1,2397.0,2397.0,0.1082
51715,80728,Vial of Magic,CraftingMaterial,Ascended,1721.0,9836.0,3410.0,12798.0,1991.0,1389.0,...,150.0,2024-08-09T11:48:25.597Z,7,198,True,150.0,7,1042.0,7294.0,0.1060
7243,9567,18 Slot Craftsman's Bag,Bag,Fine,734.0,21220.0,720.0,27592.0,122.0,278.0,...,38.0,2024-08-09T11:48:13.572Z,1,39,True,38.0,1,2233.0,2233.0,0.1052
31536,44950,Superior Sigil of Malice,UpgradeComponent,Exotic,822.0,81306.0,223.0,105500.0,439.0,502.0,...,64.0,2024-08-09T11:48:22.753Z,3,71,True,64.0,3,8369.0,25107.0,0.1029


In [ ]:
columns_selected = [
    'id', 
    'name', 
    'type', 
    'rarity', 
    'buy_quantity', 
    'buy_price',
    'sell_quantity', 
    'sell_price', 
    '7d_buy_sold', 
    '7d_sell_sold',
    '1d_buy_sold', 
    '1d_sell_sold', 
    'lastUpdate', 
    # 'daily',
    # '1d_sell_sold_avg', 
    # '1d_lower_avg', 
    # '1d_sell_sold_mod', 
    'daily_mod',
    'profit', 
    'total_profit', 
    'ROI'
]

In [29]:
def calculate_roi(sell_price, buy_price):

    if buy_price != 0:
        roi = round((sell_price * 0.85 - buy_price) / buy_price, 4)
    else:
        roi = 0
    
    return roi

In [30]:
data_transformed = data.copy()
data_transformed['daily'] = data_transformed['1d_sell_sold'].apply(
    lambda x: math.floor(x * 0.05)
)
data_transformed['1d_sell_sold_avg'] = data_transformed['7d_sell_sold'].apply(
    lambda x: math.floor(x / 7) 
)
data_transformed['1d_lower_avg'] = (data_transformed['1d_sell_sold'] < data_transformed['1d_sell_sold_avg'])

data_transformed['1d_sell_sold_mod'] = data_transformed.apply(
    lambda row: min(row['1d_sell_sold'], row['1d_sell_sold_avg']),
    axis=1
)
data_transformed['daily_mod'] = data_transformed['1d_sell_sold_mod'].apply(
    lambda x: math.floor(x * 0.05)
)


data_transformed['profit'] = round(data_transformed['sell_price'] * 0.85 - data_transformed['buy_price'])
data_transformed['total_profit'] = round(data_transformed['profit'] * data_transformed['daily_mod'])
data_transformed['ROI'] = data_transformed.apply(
    lambda row: calculate_roi(row['sell_price'], row['buy_price']),
    axis=1
)


# (data_transformed['sell_price'] * 0.85 - data_transformed['buy_price'])/data_transformed['buy_price']


data_transformed.head()

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,...,1d_buy_sold,lastUpdate,daily,1d_sell_sold_avg,1d_lower_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,ROI
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-09-24T08:00:01.483Z,0,0,False,0.0,0,0.0,0.0,0.0000
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
3,24,Sealed Package of Snowballs,Consumable,Basic,82969.0,88.0,153151.0,127.0,2497.0,129.0,...,97.0,2024-08-09T10:20:13.740Z,27,356,False,356.0,17,20.0,340.0,0.2267
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2023-05-11T18:00:01.806Z,0,0,False,0.0,0,0.0,0.0,0.0000


In [52]:
field_string =f"fields={','.join(fields)}" 
field_string

'fields=id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate'

In [53]:
filter_string = f"filter={','.join(filters)}"
filter_string

'filter='

In [54]:
query_url = f"{DATA_GW2_URL}?{field_string}&{filter_string}"
# query_url = f"{DATA_GW2_URL}?{field_string}"
query_url

'https://api.datawars2.ie/gw2/v1/items/csv?fields=id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate&filter='

In [57]:
data = pd.read_csv(query_url).fillna(0)
data.head()

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-24T08:00:01.483Z
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
3,24,Sealed Package of Snowballs,Consumable,Basic,83008.0,88.0,153150.0,127.0,2497.0,129.0,542.0,122.0,1025.0,97.0,2024-08-09T08:48:09.914Z
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-11T18:00:01.806Z


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67146 entries, 0 to 67145
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67146 non-null  int64  
 1   name               67146 non-null  object 
 2   type               67146 non-null  object 
 3   rarity             67146 non-null  object 
 4   buy_quantity       67146 non-null  float64
 5   buy_price          67146 non-null  float64
 6   sell_quantity      67146 non-null  float64
 7   sell_price         67146 non-null  float64
 8   7d_sell_sold       67146 non-null  float64
 9   7d_sell_price_avg  67146 non-null  float64
 10  1d_sell_sold       67146 non-null  float64
 11  1d_sell_price_avg  67146 non-null  float64
 12  lastUpdate         67146 non-null  object 
dtypes: float64(8), int64(1), object(4)
memory usage: 6.7+ MB
